<a href="https://colab.research.google.com/github/KhushMody/Deep_Learning_101/blob/master/Raw_L_Layer_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAW L-LAYER NEURAL NETWORK

### IMPORTING PACKAGES


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import h5py

### HELPER FUNCTIONS

In [ ]:
def sigmoid(x):
  s = 1/(1+np.exp(-x))
  return s

def relu(x):
  s = np.maximum(0,x)
  return s

def relu_backward(dA, cache):
  Z = cache
  dZ = np.array(dA, copy = True)
  dZ[Z <= 0] = 0
  return dZ

def sigmoid_backward(dA, cache):
  Z = cache
  s = 1/(1+np.exp(-Z))
  dZ = dA * s * (1-s)
  return dZ

### INITIALIZATION

In [ ]:
def initialize_parameters(layers_dims):
  parameters = {}
  L = len(layers_dims)

  for l in range(1,L):
    parameters['W' + str(l)] = np.random.randn(layers_dims[l],layers_dims[l-1])*np.sqrt(2/layers_dims[l-1])
    parameters['b' + str(l)] = np.random.randn(layers_dims[l],1)
  
  return parameters

### FORWARD PROPOGATION


1.LINEAR FORWARD

In [ ]:
def linear_forward(A, W, b):
  Z = np.dot(W,A) + b
  cache = (A,W,b)

  return Z,cache

2.LINEAR ACTIVATION FORWARD

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
  if activation == "relu":
    Z,linear_cache = linear_forward(A_prev, W, b)
    A = relu(Z)
    activation_cache = Z
  if activation == "sigmoid":
    Z,linear_cache = linear_forward(A_prev, W, b)
    A = sigmoid(Z)
    activation_cache = Z
  cache = (linear_cache, activation_cache)
  return A,cache

3.L-LAYER FORWARD PROPOGATION

In [ ]:
def l_layer_forward_propogation(X,parameters):
  A = X
  L = len(parameters) // 2
  caches = []

  for l in range(1,L):
    A_prev = A
    A,cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = 'relu')
    caches.append(cache)

  Yhat,cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(l)], activation = "sigmoid")
  caches.append(cache)

  return AL,caches

###  COST FUNCTION

In [ ]:
def compute_cost(AL, Y):
  m = Y.shape[1]
  cost = -(np.dot(Y,np.log(AL).T)+np.dot((1-Y),(np.log(1-AL)).T))/m
  cost = np.squeeze(cost)

  return cost

### BACKWARD PROPOGATION

1.LINEAR BACKWARD

In [ ]:
def linear_backward(dZ, cache):
  A_prev, W, b = cache
  m = A_prev.shape[1]

  dW = (1/m) * (np.dot(dZ,A_prev.T))
  db = (1/m) * (np.sum(dZ, axis = 1, keepdims = True))
  dA_prev = np.dot(W.T, dZ)

  return dA_prev, dW, db

2.LINEAR ACTIVATION BACKWARD

In [ ]:
def linear_activation_backward(dA, cache, activation):
  linear_cache, activation_cache = cache
  if activation == "relu":
    dZ = relu_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  if activation == "sigmoid":
    dZ = sigmoid_backward(dA, activation_cache)
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

  return dA_prev, dW, db

L-LAYER BACK PROPOGATION

In [ ]:
def L_layer_back_propogation(AL, Y, caches):
  grads = {}
  L = len(caches)
  m = AL.shape[1]
  Y = Y.reshape(AL.shape)

  dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
  current_cache = caches[L-1]
  grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")

  for l in  reversed(range(L-1)):
    current_cache = caches[l]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l+1)], current_cache, activation = "relu")
    grads["dA" + str(l)] = dA_prev_temp
    grads["dW" + str(l)] = dW_temp
    grads["db" + str(l)] = db_temp

  return grads

### PARAMETER UPDATION

In [ ]:
def update_parameters(parameters, grads, learning_rate):
  L = len(parameters) // 2
  for l in range(L):
    parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * grads["dW" + str(l+1)])
    parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * grads["db" + str(l+1)])
  
  return parameters